In [38]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [39]:
elo_hard=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2021-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)

elo_clay=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date not like '{time_now_formatted}'",
    con=devengine,)


In [40]:
elo_data_hard =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' and resulted like 'False' --and Elo_Fav_Odds>1.19 --and Winner_Rank BETWEEN 1 AND 200 --and Winner_Rank<1000",
    con=devengine,)


elo_data_clay =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' --and Elo_Fav_Odds>1.19",
    con=devengine,)

In [41]:
def get_player_record(player,opponent_rank,history,time,opponent,rank):
    player_history_top50=pd.DataFrame()
    player_history_top100=pd.DataFrame()
    player_history=pd.DataFrame()
    if opponent_rank<50:
        player_history_top50=history[(((history['Fav']==player)&(history['Dog_Rank']<50))|((history['Dog']==player)&(history['Fav_Rank']<50)))]
        #if player_history_top50.empty==False and (len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))  > 0.5:
            
            #print(time,player,"Top50",len(player_history_top50))
            #print(len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))        
    if opponent_rank<100:
        player_history_top100=history[(((history['Fav']==player)&(history['Dog_Rank']<100))|((history['Dog']==player)&(history['Fav_Rank']<100)))]
        #if player_history_top100.empty==False and (len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   > 0.5:     
        #    print(time,player,"Top100",len(player_history_top100))
        #    print(len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   
            
    else:
        opponent_rank_low=opponent_rank-50
        opponent_rank_high=opponent_rank+50
        player_history=history[((history['Fav']==player)&((history['Dog_Rank']>opponent_rank_low)&(history['Dog_Rank']<opponent_rank_high)))|
        ((history['Dog']==player)&((history['Fav_Rank']>opponent_rank_low)&(history['Fav_Rank']<opponent_rank_high)))]
        #if player_history.empty==False:
        #    if (len(player_history[player_history['Winner']==player])/len(player_history)) > 0.5 and len(player_history) >5:
        #        print(time,player,opponent_rank_low,opponent_rank_high,len(player_history))
        #        print(len(player_history[player_history['Winner']==player])/len(player_history))       
    
    return player_history,player_history_top100,player_history




In [42]:
def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0


In [43]:
result_df=pd.DataFrame()
for _,row in elo_data_hard.sort_values(by='Time').iterrows():
    low_limit=50
    high_limit=50

    percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games <10 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)

    low_limit=50
    high_limit=50
    percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games2 <5 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)       
        
    if ((percent >= 0.5 and percent2 < 0.4)) and games > 4 and games2 > 4:
        # if  (percent2 < 5)| (percent < 5):
        temp_df = pd.DataFrame(
            {   "Time":[row.Time],
                "Fav": [row.Fav],
                "Fav_Record": ["{:.0%}".format(percent)],
                "Fav_Games": [games],
                "Dog": [row.Dog],
                "Dog_Record": ["{:.0%}".format(percent2)],
                "Dog_Games": [games2],
            }
        )

        result_df=pd.concat([result_df,temp_df])
        #print(row.Time, row.Fav, row.Fav_Odds, row.Dog, row.Dog_Odds)
        #print(
        #    "{:.0%}".format(percent),
        #    f"({games} Games)",
        #    "{:.0%}".format(percent2),
        #    f"({games2} Games)",
        #)
result_df      


In [44]:
serving=pd.read_excel('servers_today.xlsx')
serving=serving.drop(columns='Time')
pd.merge(result_df,serving,how='left',left_on=['Fav','Dog'],right_on=['Fav','Dog'])

,Time,Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
0,19:10,Lorenzo Musetti,60%,20,Nikoloz Basilashvili,27%,15,77,21,73,23


In [50]:
pd.read_sql_query("Select * FROM AllMatches --where date = '2022-11-02'",con=devengine)

,Date,Sex,Tournament,Player_1,Player_2,Player_1_Odds,Player_2_Odds,Surface,Player_1_Rank,Player_2_Rank
0,2020-01-03,Mens,ATP Cup9,Taylor Fritz,Viktor Durasovic,1.09,7.14,Hard,31.0,329.0
1,2020-01-03,Mens,ATP Cup9,Steve Darcis,Alexander Cozbinov,1.04,11.53,Hard,200.0,816.0
2,2020-01-03,Mens,ATP Cup9,Nick Kyrgios,Jan Lennard Struff,1.45,2.70,Hard,29.0,35.0
3,2020-01-03,Mens,ATP Cup9,Karen Khachanov,Stefano Travaglia,1.14,5.45,Hard,17.0,82.0
4,2020-01-03,Mens,ATP Cup9,Grigor Dimitrov,Daniel Evans,1.49,2.60,Hard,20.0,42.0
...,...,...,...,...,...,...,...,...,...,...
42364,2022-11-01,Womens,Midland WTA5,Nao Hibino,Shuai Zhang,3.42,1.30,Hard,144.0,24.0
42365,2022-11-01,Womens,Midland WTA5,Robin Montgomery,Katherine Sebov,1.34,3.18,Hard,257.0,229.0
42366,2022-11-01,Womens,Midland WTA5,Coco Vandeweghe,Magdalena Frech,2.37,1.55,Hard,125.0,114.0
42367,2022-11-01,Womens,Midland WTA275,Robin Montgomery,Diana Shnaider,1.60,2.24,Hard,257.0,186.0
